# Hyperdimensional Centroid Language classification

Paper experiment setup: While in this study, in order to enable training of SOMs, each language corpus was divided into samples where the length of each sample was set to 1, 000 symbols. The data for each language was preprocessed such that the text included only lower case letters and spaces. All punctuation was removed. Lastly, all text used the 26-letter ISO basic Latin alphabet, i.e., the alphabet for both training and test data was the same and it included 27 symbols. 

In [1]:
import re
import numpy as np

In [2]:
alphabet = 'abcdefghijklmnopqrstuvwxyz '  # 27 symbols including space
alphabet_size = len(alphabet)
hd_dimension1 = 100  
hd_dimension2 = 1000

In [3]:
def ngrams(text, n=3):
    ngrams = []
    for i in range(len(text) - n + 1):
        ngram = text[i:i + n]
        ngrams.append(ngram)
    
    return ngrams


In [4]:
def preprocess(text):
    # Lowercase the text
    text = text.lower()
    text = re.sub(r'[^a-z ]', '', text)
    return text

example1 = '4	Ah, dernier truc : l’idéal est de vider la cuisine d’un trait en prenant bien le temps de ranger et non pas petit à petit mais dans l’urgence comme j’ai fait car cela devient vite le bazar.'
processed_text1 = preprocess(example1)

print("Original text:")
print(example1)
print("\nPreprocessed text:")
print(processed_text1)


Original text:
4	Ah, dernier truc : l’idéal est de vider la cuisine d’un trait en prenant bien le temps de ranger et non pas petit à petit mais dans l’urgence comme j’ai fait car cela devient vite le bazar.

Preprocessed text:
ah dernier truc  lidal est de vider la cuisine dun trait en prenant bien le temps de ranger et non pas petit  petit mais dans lurgence comme jai fait car cela devient vite le bazar


In [5]:
ngrams_example1 = ngrams(processed_text1)

In [6]:
def initialize_hd_vectors(ngrams, d):
    hd_vectors = {}
    for ngram in ngrams:
        hd_vectors[ngram] = np.random.choice([-1, 1], size=(d,))
    return hd_vectors


In [7]:
def init_memory(alphabet_size, hd_dimension):
    H = np.random.choice([-1, 1], size=(hd_dimension, alphabet_size))
    return H

def get_hd_vector(symbol_index, item_memory):
    # Retrieve the HD vector for the given symbol index from item memory H
    return item_memory[:, symbol_index]


In [8]:
# Initialize item memory
item_memory = init_memory(alphabet_size, hd_dimension1)

# Retrieve HD vector for a specific symbol (e.g., 'a')
symbol_index = alphabet.index('a')
hd_vector_a = get_hd_vector(symbol_index, item_memory)

print(f"Item memory (H):\n{item_memory.shape}")
print(f"\nHD vector for symbol 'a' (H[a]):\n{hd_vector_a.shape}")

Item memory (H):
(100, 27)

HD vector for symbol 'a' (H[a]):
(100,)


In [9]:
def rho(hd_vector, times, shift=5):
    if times == 0:
        return hd_vector
    else:
        # Rotate the hd_vector by 'shift' positions
        return rho(np.roll(hd_vector, shift), shift, times - 1)


# forming HD vector of an n-gram

From paper: for the trigram ‘cba’ will be mapped to its HD vector as follows:
ρ
1
(Hc)  ρ
2
(Hb)  ρ
3
(Ha) the process of forming HD vector of an n-gram can be formalized as follows

In [10]:
def ngram_HD(ngram, hd_dimension):
    ngram_hd = np.ones((hd_dimension,) )
    for i in range(len(ngram)):
        sym_idx = alphabet.index(ngram[i])
        hd_vector = get_hd_vector(sym_idx, item_memory)
        # apply i+1 times rho
        hd_vector = rho(hd_vector, i+1)
        ngram_hd = ngram_hd*hd_vector
    return ngram_hd

ngram_example = 'cba'
example_output = ngram_HD(ngram_example, 100)
print('cab hd vector:', example_output)

cab hd vector: [ 1. -1.  1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1.  1.  1.
  1. -1. -1. -1.  1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1. -1.  1.  1.
 -1. -1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1.
  1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1. -1.  1. -1.
  1.  1.  1.  1. -1. -1. -1.  1.  1. -1.]


# mapping the whole n-gram statistics s

In [11]:
from collections import defaultdict

def ngram_stat_h(ngrams, hd_dimension, n=3, alphabet_size=alphabet_size):

    h = np.zeros((hd_dimension,))
    # Initialize vector s
    num_ngrams = alphabet_size ** n
    s_i = np.zeros(num_ngrams)

    # Create a mapping from n-gram to index in vector s
    ngram_to_index = defaultdict(lambda: len(ngram_to_index))
    
    for ngram in ngrams:
        index = ngram_to_index[ngram]
        s_i[index] += 1
        
    for ngram in ngrams:    
        ngram_hd = ngram_HD(ngram, hd_dimension)
        h += s_i[ngram_to_index[ngram]]*ngram_hd
    return h



In [12]:
example1_h_stat = ngram_stat_h(ngrams_example1, hd_dimension1)
print('a short sentence example n-gram statistics:\n', example1_h_stat)

a short sentence example n-gram statistics:
 [ -8.   8. -32.   8.   0.   0. -22. -38.  22. -20. -22. -20.  -6.  -2.
 -22. -20.  38.  16.  24.  16. -24. -52.  48.  14.  14.  50. -20. -14.
  12. -24.  -4.   0. -10. -16. -16. -10.   6.  58. -24.  32.  -2. -14.
  24.  28. -52.  22. -24. -68. -26. -10. -38.   4. -54.  46.  12.  42.
  20. -36.  26.   6. -38.  54.  12. -18. -16. -26.  22.   2.  10.  66.
  22.  46.  12.  26. -16. -32.  12.  26.  10.  -8. -10.  -8.  32. -14.
 -40.   6.   8. -40. -58. -62.  46.  72. -84.  -8. -12.  44.  26.  16.
   6. -34.]


In [27]:
l2_norm_example1 = np.linalg.norm(example1_h_stat)

normalized_array = example1_h_stat / l2_norm_example1

print("\nL2 norm of the array:")
print(l2_norm_example1)
print("\nNormalized array:")
print(normalized_array)



L2 norm of the array:
303.84864653310535

Normalized array:
[-0.0263289   0.0263289  -0.10531559  0.0263289   0.          0.
 -0.07240447 -0.12506227  0.07240447 -0.06582224 -0.07240447 -0.06582224
 -0.01974667 -0.00658222 -0.07240447 -0.06582224  0.12506227  0.0526578
  0.07898669  0.0526578  -0.07898669 -0.17113784  0.15797339  0.04607557
  0.04607557  0.16455561 -0.06582224 -0.04607557  0.03949335 -0.07898669
 -0.01316445  0.         -0.03291112 -0.0526578  -0.0526578  -0.03291112
  0.01974667  0.19088451 -0.07898669  0.10531559 -0.00658222 -0.04607557
  0.07898669  0.09215114 -0.17113784  0.07240447 -0.07898669 -0.22379563
 -0.08556892 -0.03291112 -0.12506227  0.01316445 -0.17772006  0.15139116
  0.03949335  0.13822671  0.06582224 -0.11848004  0.08556892  0.01974667
 -0.12506227  0.17772006  0.03949335 -0.05924002 -0.0526578  -0.08556892
  0.07240447  0.00658222  0.03291112  0.21721341  0.07240447  0.15139116
  0.03949335  0.08556892 -0.0526578  -0.10531559  0.03949335  0.08556892

# 21 languages centroid HD

In [31]:
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = file.readlines()
    return sentences

In [37]:
def pick_random_sentences(sentences, num_sentences=1000):
    return random.sample(sentences, min(num_sentences, len(sentences)))

In [63]:
import os
import random

folder_path = './news'

def HD_centroid_1(hd_dimension):
    # Initialize item memory
    item_memory = init_memory(alphabet_size, hd_dimension)

    HD_centroids = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                preprocessed_text = preprocess(text)
                ngrams_text = ngrams(preprocessed_text)
                h_stat = ngram_stat_h(ngrams_text, hd_dimension)
                # L2 normalize h
                l2_norm = np.linalg.norm(h_stat)
                normed_h = h_stat / l2_norm
                HD_centroids[filename.split('_')[0]] = normed_h
    return HD_centroids

def HD_centroid(hd_dimension):
    # Initialize item memory
    item_memory = init_memory(alphabet_size, hd_dimension)

    HD_centroids = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            centroid = []
            file_path = os.path.join(folder_path, filename)
            sentences = read_sentences(file_path)
            sentences = pick_random_sentences(sentences, num_sentences=1000)
            for sentence in sentences:
                preprocessed_sentence = preprocess(sentence)
                ngrams_sentence = ngrams(preprocessed_sentence)
                h_stat = ngram_stat_h(ngrams_sentence, hd_dimension)
                # L2 normalize h
                epsilon = 1e-4
                l2_norm = np.linalg.norm(h_stat)
                normed_h = h_stat / (l2_norm + epsilon)
                centroid.append(normed_h)
            mean = np.mean(np.array(centroid), axis=0)
            HD_centroids[filename.split('_')[0]] = mean
            print(f'Language: {filename.split('_')[0]} HD centroid is generated')
    return HD_centroids

In [64]:
HD_centroids_100dimension = HD_centroid(hd_dimension1)

Language: bul HD centroid is generated
Language: ces HD centroid is generated
Language: dan HD centroid is generated
Language: deu HD centroid is generated
Language: ell HD centroid is generated
Language: eng HD centroid is generated
Language: est HD centroid is generated
Language: fin HD centroid is generated
Language: fra HD centroid is generated
Language: hun HD centroid is generated
Language: ita HD centroid is generated
Language: lav HD centroid is generated
Language: lit-lit HD centroid is generated
Language: nld HD centroid is generated
Language: pol HD centroid is generated
Language: por HD centroid is generated
Language: ron HD centroid is generated
Language: slk HD centroid is generated
Language: slv HD centroid is generated
Language: spa HD centroid is generated
Language: swe HD centroid is generated


In [65]:
import pickle

with open('HD_centroids_100dimension.pkl', 'wb') as f:
    pickle.dump(HD_centroids_100dimension, f)

print("Dictionary saved to HD_centroids_100dimension.pkl")


Dictionary saved to HD_centroids_100dimension.pkl


In [66]:
print('Italian HD cenroid vector:\n')
HD_centroids_100dimension['ita']

Italian HD cenroid vector:



array([ 0.01090108,  0.08566247, -0.02646928, -0.01193203,  0.00256517,
       -0.02918485,  0.01382562, -0.01469623,  0.0321326 , -0.03872871,
        0.06623449, -0.01312025, -0.00586137, -0.00469706,  0.02872543,
        0.00489514,  0.01251363, -0.0038832 ,  0.01833483,  0.00967444,
       -0.01364578,  0.01118   ,  0.01397018, -0.02137935, -0.01227579,
        0.02478084, -0.03249292, -0.00836385,  0.04203523, -0.01865708,
        0.00075424,  0.01881816,  0.03630089,  0.02538361, -0.01925239,
        0.00211136,  0.00371725,  0.03956649,  0.00446612, -0.0318339 ,
        0.00283472, -0.04907745,  0.00483597,  0.00151961, -0.02528929,
        0.00447018, -0.03767264,  0.00542936, -0.00886847, -0.02129426,
        0.04042387, -0.0055643 , -0.03544944,  0.01644598, -0.04382651,
       -0.01559918, -0.007228  , -0.01165462,  0.09678338,  0.0129941 ,
        0.00857646, -0.00732188, -0.02434546,  0.03468858,  0.00429537,
       -0.04575419, -0.02184909,  0.03261962,  0.0522965 ,  0.00

# Test phase using Europarl Parallel Corpus

In [67]:
# Load the dictionary from the file
with open('HD_centroids_100dimension.pkl', 'rb') as f:
    loaded_HD_centroid = pickle.load(f)




In [68]:
test_path = './finaltest'

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    
def cosine_classify(test_path, centroids, hd_dimension):
    confusion_matrix = {}
    for filename in os.listdir(test_path):
        file_path = os.path.join(test_path, filename)
        sentences = read_sentences(file_path)
        sentences = pick_random_sentences(sentences, num_sentences=100)
        confusion_matrix[filename] = []
        
        for sentence in sentences:
            preprocessed_sentence = preprocess(sentence)
            ngrams_sentence = ngrams(preprocessed_sentence)
            h_stat = ngram_stat_h(ngrams_sentence, hd_dimension)
            # L2 normalize h
            l2_norm = np.linalg.norm(h_stat)
            epsilon = 1e-4
            normed_h = h_stat / (l2_norm + epsilon)
            
            for label, centroid in centroids.items():
                similarity[label] = cosine_similarity(normed_h, centroid)
    return similarity


In [69]:
similarity = cosine_classify(test_path, HD_centroids_100dimension, hd_dimension1)

In [70]:
pred = max(similarity, key=similarity.get)
pred

'swe'